In [1]:
import sys
sys.path.append('../../../')
sys.path.append('/home/sheida.rahnamai/GIT/HDN/')
import torch
import pickle
import matplotlib.pyplot as plt
import numpy as np
from tifffile import imread
import glob
import matplotlib.patches as patches
import hdbscan as hd
from hdbscan import prediction
from sklearn.cluster import HDBSCAN
from collections import Counter
from openTSNE import TSNE
from openTSNE import TSNEEmbedding
from openTSNE import affinity
from openTSNE import initialization
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
# %reload lib.plotting 
from lib import plotting as p
from lib import dataprep as dp
import random
import umap
from tqdm import tqdm
from lib.evaluation import FeatureExtractor
# from lib.dataloader import CustomTestDataset
from lib.dataloader import CustomDataset
import tifffile as tiff
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import os
from boilerplate import boilerplate
from importlib import reload
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix, adjusted_rand_score, normalized_mutual_info_score, homogeneity_score, completeness_score, v_measure_score

## Loading the model

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

patch_size = 64
sample_size = 350
centre_size = 4
n_channel = 32
hierarchy_level = 3
pad_size = (patch_size - centre_size) // 2
model_dir = "/group/jug/Sheida/HVAE/v01/model/"
model = torch.load(model_dir+"HVAE_best_vae.net")

### Loading train data
### Sample from different classes
### Loading test data

In [16]:
with open('/group/jug/Sheida/pancreatic beta cells/download/train/10_percent_nucleus.pickle', 'rb') as file:
    # Deserialize and retrieve the variable from the file
    loaded_data = pickle.load(file)

In [10]:
print(len(loaded_data))

1777


In [8]:
# train data

data_dir = "/group/jug/Sheida/pancreatic beta cells/download/"
keys = ['high_c1', 'high_c2', 'high_c3']

# Load source images
train_img_paths = [os.path.join(data_dir + 'train/' + key + f"/{key}_source.tif") for key in keys]
train_lbl_paths = [os.path.join(data_dir + 'train/' + key + f"/{key}_gt.tif") for key in keys]
val_img_paths = [os.path.join(data_dir + 'val/' + key + f"/{key}_source.tif") for key in keys]
val_lbl_paths = [os.path.join(data_dir + 'val/' + key + f"/{key}_gt.tif") for key in keys]

train_images = {key: tiff.imread(path) for key, path in zip(keys, train_img_paths)}
train_labels = {key: tiff.imread(path) for key, path in zip(keys, train_lbl_paths)}

val_images = {key: tiff.imread(path) for key, path in zip(keys, val_img_paths)}
val_labels = {key: tiff.imread(path) for key, path in zip(keys, val_lbl_paths)}

for key in tqdm(keys, desc='filtering out outside of the cell'):
   filtered_image, filtered_label = boilerplate._filter_slices(train_images[key], train_labels[key])
   train_images[key] = filtered_image
   train_labels[key] = filtered_label

   filtered_image, filtered_label = boilerplate._filter_slices(val_images[key], val_labels[key])
   
   val_images[key] = filtered_image
   val_labels[key] = filtered_label

# compute mean and std of the data
all_elements = np.concatenate([train_images[key].flatten() for key in keys])
data_mean = np.mean(all_elements)
data_std = np.std(all_elements)

# normalizing the data
for key in tqdm(keys, 'Normalizing data'):
   train_images[key] = (train_images[key] - data_mean) / data_std
   val_images[key] = (val_images[key] - data_mean) / data_std

train_set = CustomDataset(train_images, train_labels)
val_set = CustomDataset(val_images, val_labels)


Normalizing data: 100%|██████████| 3/3 [00:02<00:00,  1.34it/s]
Extracting patches from high_c1: 965it [00:02, 326.09it/s]
Extracting patches from high_c2: 889it [00:02, 355.70it/s]
Extracting patches from high_c3: 907it [00:02, 346.44it/s]
Extracting patches from high_c1: 109it [00:00, 377.35it/s]
Extracting patches from high_c2: 101it [00:00, 409.37it/s]
Extracting patches from high_c3: 103it [00:00, 375.83it/s]


In [ ]:

One_test_image = ['high_c4']

# Load test image
test_img_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_source.tif")
test_image = tiff.imread(test_img_path)

# Print loaded test images paths
print("Test image loaded from path:")
print(test_img_path)

# Load test ground truth images
test_gt_path = os.path.join(data_dir, One_test_image[0], f"{One_test_image[0]}_gt.tif")
test_ground_truth_image = tiff.imread(test_gt_path)

bg_indices = random.sample(train_set.patches_by_label[0],sample_size)
nucleus_indices = random.sample(train_set.patches_by_label[1],sample_size)
granule_indices = random.sample(train_set.patches_by_label[2],sample_size)
mito_indices = random.sample(train_set.patches_by_label[3],sample_size)

bg_samples, bg_cls, bg_lbl = train_set[bg_indices]
nucleus_samples, nucleus_cls, nucleus_lbl = train_set[nucleus_indices]
granule_samples, granule_cls, granule_lbl = train_set[granule_indices]
mito_samples, mito_cls, mito_lbl = train_set[mito_indices]

bg_samples = bg_samples.squeeze(1)
bg_lbl = bg_lbl.squeeze(1)
nucleus_samples = nucleus_samples.squeeze(1)
nucleus_lbl = nucleus_lbl.squeeze(1)
granule_samples = granule_samples.squeeze(1)
granule_lbl = granule_lbl.squeeze(1)
mito_samples = mito_samples.squeeze(1)
mito_lbl = mito_lbl.squeeze(1)

#test data

test_images = (test_image-data_mean)/data_std

test_slice = test_images[626]
test_slice_lbl = test_ground_truth_image[626]

Uncategorized: 60777 Nucleus: 17907 Granule: 17835 Mitochondria: 17907

In [ ]:
train_batch = torch.cat([bg_samples, nucleus_samples, granule_samples, mito_samples], dim=0)

In [ ]:
model.mode_pred = True
model.eval()
device = model.device
with torch.no_grad():
    train_batch = train_batch.to(device=device, dtype=torch.float)
    train_batch = train_batch.reshape(-1, 1, patch_size, patch_size)
    output = model(train_batch, train_batch, train_batch, model_layers=[i for i in range(hierarchy_level)])
    mu = output["mu"]

In [ ]:
print(mu[0].shape, mu[1].shape, mu[2].shape)

In [ ]:
p.plot_w_b((test_slice, 'Training patch', False),\
        (test_slice_lbl, 'Ground truth label', False),\
        plot_types=['imshow', 'imshow_l'],\
        box_size = 10
        )

In [ ]:
i, j, k, l = np.random.randint(0, sample_size, 4)
p.plot_w_b((bg_samples[i], 'Training patch', (pad_size, pad_size, centre_size)),\
        (bg_lbl[i], 'Ground truth label', (pad_size, pad_size, centre_size)),\
        (nucleus_samples[j], 'Training patch', (pad_size, pad_size, centre_size)),\
        (nucleus_lbl[j], 'Ground truth label', (pad_size, pad_size, centre_size)),\
        (granule_samples[k], 'Training patch', (pad_size, pad_size, centre_size)),\
        (granule_lbl[k], 'Ground truth label', (pad_size, pad_size, centre_size)),\
        (mito_samples[l], 'Training patch', (pad_size, pad_size, centre_size)),\
        (mito_lbl[l], 'Ground truth label', (pad_size, pad_size, centre_size)),\
        plot_types=['imshow', 'imshow_l', 'imshow', 'imshow_l', 'imshow', 'imshow_l', 'imshow', 'imshow_l']
        )

## Extracting latent space from Encoder

In [ ]:
bg_mu, nuc_mu, gra_mu, mito_mu = np.empty((0, 43008)), np.empty((0, 43008)), np.empty((0, 43008)), np.empty((0, 43008))
FE = FeatureExtractor(model, patch_size, centre_size)
for idx in range(sample_size):
    mu = FE.get_feature_maps(bg_samples[idx])
    bg_mu = np.vstack([bg_mu, mu])
    mu = FE.get_feature_maps(nucleus_samples[idx])
    nuc_mu = np.vstack([nuc_mu, mu])
    mu = FE.get_feature_maps(granule_samples[idx])
    gra_mu = np.vstack([gra_mu, mu])
    mu = FE.get_feature_maps(mito_samples[idx])
    mito_mu = np.vstack([mito_mu, mu])
train_mu = np.concatenate((bg_mu, nuc_mu, gra_mu, mito_mu), axis=0)
train_labels = np.array([0] * sample_size + [1] * sample_size + [2] * sample_size + [3] * sample_size)

In [ ]:
print(bg_mu.shape, nuc_mu.shape, gra_mu.shape, mito_mu.shape, train_mu.shape, train_labels.shape)

In [ ]:
bg_gra_dist = cdist(bg_mu, gra_mu, 'euclidean')
total_bg_gra_dist = np.mean(bg_gra_dist)
print(f"Uncategorized vs Granule distance: {total_bg_gra_dist}")

bg_nuc_dist = cdist(bg_mu, nuc_mu, 'euclidean')
total_bg_nuc_dist = np.mean(bg_nuc_dist)
print(f"Uncategorized vs Nucleus distance: {total_bg_nuc_dist}")

bg_mito_dist = cdist(bg_mu, mito_mu, 'euclidean')
total_bg_mito_dist = np.mean(bg_mito_dist)
print(f"Uncategorized vs Mitochondria distance: {total_bg_mito_dist}")

nuc_gra_dist = cdist(nuc_mu, gra_mu, 'euclidean')
total_nuc_gra_dist = np.mean(nuc_gra_dist)
print(f"Nucleus vs Granule distance: {total_nuc_gra_dist}")

nuc_mito_dist = cdist(nuc_mu, mito_mu, 'euclidean')
total_nuc_mito_dist = np.mean(nuc_mito_dist)
print(f"Nucleus vs Mitochondria distance: {total_nuc_mito_dist}")

gra_mito_dist = cdist(gra_mu, mito_mu, 'euclidean')
total_gra_mito_dist = np.mean(gra_mito_dist)
print(f"Granule vs Mitochondria distance: {total_gra_mito_dist}")

In [ ]:
distances = cdist(train_mu, train_mu, metric='euclidean')

In [ ]:
classes = ['Uncategorized', 'Nucleus', 'Granule', 'Mitochondria']

plt.figure(figsize=(20, 16))
ax = sns.heatmap(distances, cmap='viridis')

# Function to draw custom bracket annotations
def draw_custom_bracket(ax, start, end, label):
    mid = (start + end) / 2
    # Draw the bracket
    ax.plot([-0.05, -0.1], [start, start], color='black', lw=1.5)
    ax.plot([-0.1, -0.1], [start, end], color='black', lw=1.5)
    ax.plot([-0.1, -0.05], [end, end], color='black', lw=1.5)
    # Add the text label
    ax.text(-0.15, mid, label, va='center', ha='right', fontsize=12)

# Add custom brackets for each class
for i, class_name in enumerate(classes):
    start = sample_size * i
    end = sample_size * (i + 1)
    draw_custom_bracket(ax, start, end, class_name)

# Add horizontal lines to separate different classes
separation_positions = [sample_size * (i + 1) for i in range(len(classes) - 1)]
for pos in separation_positions:
    ax.axhline(pos, color='black', linestyle='--', lw=1.5)
    ax.axvline(pos, color='black', linestyle='--', lw=1.5)

# Hide the ticks
ax.set_xticks([])
ax.set_yticks([])

plt.title('Pairwise Distance Heatmap')
plt.show()

In [ ]:
def get_patch(image, labels, cls, patch_size):
    while True: 
        patch, label, y, x = dp.get_random_patch(image, labels, patch_size)
        centre = label[pad_size:pad_size+centre_size,pad_size:pad_size+centre_size]
        if cls in centre and np.unique(centre).size == 1:
                return patch, label, y, x


In [ ]:
bg_patch, bg_label, y, x = get_patch(test_slice, test_slice_lbl, 0, patch_size)
test_mu_bg = FE.get_feature_maps(bg_patch)
test_mu_bg = np.expand_dims(test_mu_bg, axis=0)

p.plot_w_b( (test_slice, "Test Image", (y, x, patch_size)),\
        (test_slice_lbl, "Ground truth label", (y, x, patch_size)),\
        (bg_patch, "Random patch", (pad_size, pad_size, centre_size)),\
        (bg_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        plot_types=['imshow', 'imshow_l', 'imshow', 'imshow_l'],
        box_size=10
        )

In [ ]:
nuc_patch, nuc_label, y, x = get_patch(test_slice, test_slice_lbl, 1, patch_size)
test_mu_nuc = FE.get_feature_maps(nuc_patch)
test_mu_nuc = np.expand_dims(test_mu_nuc, axis=0)

p.plot_w_b( (test_slice, "Test Image", (y, x, patch_size)),\
        (test_slice_lbl, "Ground truth label", (y, x, patch_size)),\
        (nuc_patch, "Random patch", (pad_size, pad_size, centre_size)),\
        (nuc_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        plot_types=['imshow', 'imshow_l', 'imshow', 'imshow_l'],
        box_size=10
        )

In [ ]:
gra_patch, gra_label, y, x = get_patch(test_slice, test_slice_lbl, 2, patch_size)
test_mu_gra = FE.get_feature_maps(gra_patch)
test_mu_gra = np.expand_dims(test_mu_gra, axis=0)

p.plot_w_b( (test_slice, "Test Image", (y, x, patch_size)),\
        (test_slice_lbl, "Ground truth label", (y, x, patch_size)),\
        (gra_patch, "Random patch", (pad_size, pad_size, centre_size)),\
        (gra_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        plot_types=['imshow', 'imshow_l', 'imshow', 'imshow_l'],
        box_size=10
        )

In [ ]:
mito_patch, mito_label, y, x = get_patch(test_slice, test_slice_lbl, 3, patch_size)
test_mu_mito = FE.get_feature_maps(mito_patch)
test_mu_mito = np.expand_dims(test_mu_mito, axis=0)

p.plot_w_b( (test_slice, "Test Image", (y, x, patch_size)),\
        (test_slice_lbl, "Ground truth label", (y, x, patch_size)),\
        (mito_patch, "Random patch", (pad_size, pad_size, centre_size)),\
        (mito_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        plot_types=['imshow', 'imshow_l', 'imshow', 'imshow_l'],
        box_size=10
        )

## t-SNE

In [ ]:
tsne_3d = TSNE(
    n_components=3,
    random_state=42,
    learning_rate="auto",
    metric="euclidean",
    n_iter=10000,
).fit(train_mu)

In [ ]:
%matplotlib widget
colors = [
        (0.8, 0.8, 0.8),               # white for label -1
        (1, 0.8, 0.9),          # pink for label 0
        (230/255, 159/255, 0),  # orange for label 1
        (0.5803921568627451, 0.403921568627451, 0.7411764705882353), # violet for label 2
        (0, 158/255, 115/255),  # bluish green for label 3
        
    ]

# Create the colormap
cmap = mcolors.ListedColormap(colors)
# Define the bounds and normalization
bounds = [-1, 0, 1, 2, 3, 4]    
norm = mcolors.BoundaryNorm(bounds, cmap.N)
# Create a 3D scatter plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
label = ['Background', 'Nucleus', 'Granule', 'Mito']
# Plot the data points with true labels
for i in range(4):
    cluster_points = tsne_3d[train_labels == i]
    ax.scatter(cluster_points[:, 0], cluster_points[:, 1], cluster_points[:, 2], label=label[i], c=sample_size*[i], cmap=cmap, norm=norm)

ax.set_title("3D t-SNE Visualization of High-Dimensional Data")
ax.set_xlabel("t-SNE Component 1")
ax.set_ylabel("t-SNE Component 2")
ax.set_zlabel("t-SNE Component 3")
ax.legend()
plt.show()


In [ ]:
tsne_3d = TSNE(
    n_components=3,
    perplexity=40,
    random_state=42,
    learning_rate="auto",
    metric="cosine",
    n_iter=10000,
).fit(train_mu)

In [ ]:
colors = [
        (0.8, 0.8, 0.8),               # white for label -1
        (1, 0.8, 0.9),          # pink for label 0
        (230/255, 159/255, 0),  # orange for label 1
        (0.5803921568627451, 0.403921568627451, 0.7411764705882353), # violet for label 2
        (0, 158/255, 115/255),  # bluish green for label 3
        
    ]

# Create the colormap
cmap = mcolors.ListedColormap(colors)
# Define the bounds and normalization
bounds = [-1, 0, 1, 2, 3, 4]    
norm = mcolors.BoundaryNorm(bounds, cmap.N)
# Create a 3D scatter plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
label = ['Background', 'Nucleus', 'Granule', 'Mito']
# Plot the data points with true labels
for i in range(4):
    cluster_points = tsne_3d[train_labels == i]
    ax.scatter(cluster_points[:, 0], cluster_points[:, 1], cluster_points[:, 2], label=label[i], c=sample_size*[i], cmap=cmap, norm=norm)

ax.set_title("3D t-SNE Visualization of High-Dimensional Data")
ax.set_xlabel("t-SNE Component 1")
ax.set_ylabel("t-SNE Component 2")
ax.set_zlabel("t-SNE Component 3")
ax.legend()
plt.show()


In [ ]:
tsne = []
tsne = TSNE(
    n_components=2,
    perplexity=50,
    random_state=42,
    learning_rate="auto",
    metric="cosine",
    n_iter=10000,
).fit(train_mu)
bg_tsne = tsne[:sample_size]
nuc_tsne = tsne[sample_size:2*sample_size]
gra_tsne = tsne[2*sample_size:3*sample_size]
mito_tsne = tsne[3*sample_size:]

In [ ]:
point_size = 10
p.plot_w_b(([bg_tsne, nuc_tsne, gra_tsne, mito_tsne], "TSNE",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size)]),\
        plot_types=['scatter'],box_size=10
        )

In [ ]:
# Perform K-means clustering
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(train_mu)

# Create a DataFrame with tSNE results and cluster labels
kmeans_df = pd.DataFrame(tsne, columns=['t-SNE c1', 't-SNE c2'])
kmeans_df['cluster'] = cluster_labels

plt.figure(figsize=(12, 8))
sns.scatterplot(data=kmeans_df, x='t-SNE c1', y='t-SNE c2', hue='cluster', palette='Set2', s=100, alpha=0.7)
plt.title('t-SNE of Latent Features with K-means Clusters')
plt.show()

In [ ]:
Counter(cluster_labels)

In [ ]:
true_labels = np.repeat(np.arange(4), sample_size)  # Replace with your actual labels
conf_matrix = confusion_matrix(true_labels, cluster_labels)
# Visualize the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(cluster_labels), yticklabels=np.unique(true_labels))
plt.xlabel('Predicted Clusters')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
test_tsne = tsne.transform(test_mu_bg)
point_size = 5
p.plot_w_b( (bg_patch, "Test patch", (pad_size, pad_size, centre_size)),\
       (bg_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_tsne, nuc_tsne, gra_tsne, mito_tsne, test_tsne], "TSNE - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([0],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
test_tsne = tsne.transform(test_mu_nuc)
point_size = 5
p.plot_w_b( (nuc_patch, "Test patch", (pad_size, pad_size, centre_size)),\
        (nuc_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_tsne, nuc_tsne, gra_tsne, mito_tsne, test_tsne], "TSNE - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([1],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
test_tsne = tsne.transform(test_mu_gra)
point_size = 5
p.plot_w_b( (gra_patch, "Test patch", (pad_size, pad_size, centre_size)),\
        (gra_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_tsne, nuc_tsne, gra_tsne, mito_tsne, test_tsne], "TSNE - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([2],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
test_tsne = tsne.transform(test_mu_mito)
point_size = 5
p.plot_w_b( (mito_patch, "Test patch", (pad_size, pad_size, centre_size)),\
        (mito_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_tsne, nuc_tsne, gra_tsne, mito_tsne, test_tsne], "TSNE - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([3],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

## UMAP

In [ ]:
reducer = umap.UMAP(n_components=2, random_state=42)
reducer.fit(train_mu)
umap_train = reducer.transform(train_mu)
bg_umap = umap_train[:sample_size]
nuc_umap = umap_train[sample_size:2*sample_size]
gra_umap = umap_train[2*sample_size:3*sample_size]
mito_umap = umap_train[3*sample_size:]
# umap_test_bg = reducer.transform(test_mu_bg)
# umap_test_nuc = reducer.transform(test_mu_nuc)
# umap_test_gra = reducer.transform(test_mu_gra)
# umap_test_mito = reducer.transform(test_mu_mito)


In [ ]:
point_size = 5
p.plot_w_b( 
        ([bg_umap, nuc_umap, gra_umap, mito_umap], "UMAP",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size)]),\
        plot_types=[ 'scatter'],
        )

In [ ]:
p.plot_w_b( (bg_patch, "Test patch", (pad_size, pad_size, centre_size)),\
       (bg_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_umap, nuc_umap, gra_umap, mito_umap, umap_test_bg], "UMAP - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([0],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
p.plot_w_b( (nuc_patch, "Test patch", (pad_size, pad_size, centre_size)),\
       (nuc_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_umap, nuc_umap, gra_umap, mito_umap, umap_test_nuc], "UMAP - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([1],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
p.plot_w_b( (gra_patch, "Test patch", (pad_size, pad_size, centre_size)),\
       (gra_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_umap, nuc_umap, gra_umap, mito_umap, umap_test_gra], "UMAP - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([2],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
p.plot_w_b( (mito_patch, "Test patch", (pad_size, pad_size, centre_size)),\
       (mito_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_umap, nuc_umap, gra_umap, mito_umap, umap_test_mito], "UMAP - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([3],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

## PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(train_mu)
pca_train = pca.transform(train_mu)
bg_pca = pca_train[:sample_size]
nuc_pca = pca_train[sample_size:2*sample_size]
gra_pca = pca_train[2*sample_size:3*sample_size]
mito_pca = pca_train[3*sample_size:]
# pca_test_bg = pca.transform(test_mu_bg)
# pca_test_nuc = pca.transform(test_mu_nuc)
# pca_test_gra = pca.transform(test_mu_gra)
# pca_test_mito = pca.transform(test_mu_mito)

In [ ]:
%matplotlib widget
p.plot_w_b( 
        ([bg_pca, nuc_pca, gra_pca, mito_pca], "PCA",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size)]),\
        plot_types=['scatter'], with_test=False
        )

In [ ]:
p.plot_w_b( (bg_patch, "Test patch", (pad_size, pad_size, centre_size)),\
       (bg_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_pca, nuc_pca, gra_pca, mito_pca, pca_test_bg], "PCA - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([0],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
p.plot_w_b( (nuc_patch, "Test patch", (pad_size, pad_size, centre_size)),\
       (nuc_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_pca, nuc_pca, gra_pca, mito_pca, pca_test_nuc], "PCA - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([1],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
p.plot_w_b( (gra_patch, "Test patch", (pad_size, pad_size, centre_size)),\
       (gra_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_pca, nuc_pca, gra_pca, mito_pca, pca_test_gra], "PCA - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([2],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
p.plot_w_b( (mito_patch, "Test patch", (pad_size, pad_size, centre_size)),\
       (mito_label, "Ground truth label", (pad_size, pad_size, centre_size)),\
        ([bg_pca, nuc_pca, gra_pca, mito_pca, pca_test_mito], "PCA - one test sample",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size), ([3],point_size)]),\
        plot_types=['imshow', 'imshow_l', 'scatter'], with_test=True
        )

In [ ]:
x1 = 575
x2 = 725
y1 = 175
y2 = 325
p.plot_w_b((test_slice[y1:y2, x1:x2], "Test image", False),\
            (test_slice_lbl[y1:y2, x1:x2], "GT label", False),\
            plot_types=['imshow', 'imshow_l'],
            box_size=10)

In [ ]:
h, w = test_images[i][y1:y2, x1:x2].shape
new_h, new_w = h - patch_size + 1, w - patch_size + 1
feature_maps = np.zeros((new_h, new_w))
all_patches, all_labels = dp.get_all_patches(test_slice[y1:y2, x1:x2], test_slice_lbl[y1:y2, x1:x2], patch_size, stride=1)
idx = 0
for y in tqdm(range(new_h)):
    for x in range(new_w):
        mu = FE.get_feature_maps(all_patches[idx])
        feature_maps[y][x] = FE.get_closest(train_mu, mu, sample_size)
        idx += 1

In [ ]:
# result = np.where(test_slice_lbl[y1+pad_size:y2-pad_size-3, x1+pad_size:x2-pad_size-3] != -1, feature_maps, 0)

In [ ]:
p.plot_w_b( (test_slice[y1:y2, x1:x2], "Input", (pad_size, pad_size, 87)),\
        (test_slice_lbl[y1:y2, x1:x2], "GT label", (pad_size, pad_size, 87)),\
        (test_slice[y1+pad_size:y2-pad_size-3, x1+pad_size:x2-pad_size-3], "Input cropped", False),\
        (test_slice_lbl[y1+pad_size:y2-pad_size-3, x1+pad_size:x2-pad_size-3], "GT label cropped", False),\
        (feature_maps, "Prediction", False),\
        plot_types=['imshow', 'imshow_l', 'imshow', 'imshow_l', 'imshow_l'],
        box_size=10)

In [ ]:
np.unique(feature_maps, return_counts=True)

## openTSNE Advanced usage

In [ ]:
affinities_train = affinity.PerplexityBasedNN(
    train_mu,
    perplexity=30,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
)

In [ ]:
%time init_train = initialization.pca(train_mu, random_state=42)

In [ ]:
embedding_train = TSNEEmbedding(
    init_train,
    affinities_train,
    negative_gradient_method="fft",
    n_jobs=8,
    verbose=True,
)

In [ ]:
%time embedding_train_1 = embedding_train.optimize(n_iter=250, exaggeration=12)

In [ ]:
bg_adv_tsne = embedding_train_1[:sample_size]
nuc_adv_tsne = embedding_train_1[sample_size:2*sample_size]
gra_adv_tsne = embedding_train_1[2*sample_size:3*sample_size]
mito_adv_tsne = embedding_train_1[3*sample_size:]

In [ ]:

point_size = 5
p.plot_w_b(([bg_adv_tsne, nuc_adv_tsne, gra_adv_tsne, mito_adv_tsne], "TSNE - Advanced usage",\
                 [([0]*sample_size,point_size), ([1]*sample_size,point_size), ([2]*sample_size,point_size),\
                   ([3]*sample_size,point_size)]),\
        plot_types=['scatter'], box_size=5, with_test=False
        )